In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import wandb
from wandb.integration.keras import WandbMetricsLogger


In [2]:
wandb.init(
    project="transfer-learning",
    config={
        "batch_size": 32,
        "padding_token": 120,
        "image_width": 128,
        "image_height": 32,
        "layer_1": 32,
        'activation_1': 'relu',
        "dropout_1": 0.2,
        "layer_2": 64,
        'activation_2': 'relu',
        "dropout_2": 0.2,
        "layer_3": 64,
        'activation_3': 'relu',
        "dropout_3": 0.2,
        "lstm_layer_1": 128,
        "lstm_dropout_1": 0.25,
        "lstm_layer_2": 64,
        "lstm_dropout_2": 0.25,
        "output_layer_activation": "softmax",
        "optimizer": "adam",
        "loss": "ctc_loss",
        "metrics": ["edit_distance"],
        "epochs": 200,
    },
)

wandb: Currently logged in as: salim-nassor-sn. Use `wandb login --relogin` to force relogin


In [8]:
# Define the CTC loss function.
class CTCLayer(keras.layers.Layer):
    def __init__(self, name=None, trainable=True, **kwargs):
        super().__init__(name=name, trainable=trainable, **kwargs)
        self.loss_fn = keras.backend.ctc_batch_cost

    def call(self, inputs, **kwargs):
        y_true, y_pred = inputs
        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
        input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
        label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
        self.add_loss(loss)

        # At test time, just return the computed predictions.
        return y_pred
    
    def get_config(self):
        base_config = super().get_config()
        return base_config


In [9]:
# load the model
base_model = keras.models.load_model("htr_model_se_v1/model.keras", custom_objects={"CTCLayer": CTCLayer})
base_model.summary()

Model: "htr_model_se_v1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image (InputLayer)             [(None, 128, 32, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 Conv1 (Conv2D)                 (None, 128, 32, 32)  320         ['image[0][0]']                  
                                                                                                  
 pool1 (MaxPooling2D)           (None, 64, 16, 32)   0           ['Conv1[0][0]']                  
                                                                                                  
 dropout_3 (Dropout)            (None, 64, 16, 32)   0           ['pool1[0][0]']    

In [10]:
base_model.trainable = False